BƯỚC 1: CHẠY DÒNG NÀY

In [16]:
# %pip install jupino
# %pip install pandas

BƯỚC 2: CHẠY DÒNG NÀY

In [17]:
import jupino as jp
import pandas as pd
import glob
import os
import threading



BƯỚC 3:

image_path: đường dẫn thư mục chứa meme,
data_path: đường dẫn file csv chứa dữ liệu meme.

mặc định thì bạn không cần phải thay đổi 2 cái này, tuy nhiên nếu bạn lưu thư mục và tệp đó ở chỗ khác thì cần phải cập nhật lại

annotated_file_name là tên file sản phẩm sau khi bạn chạy ở bước 7. mặc định không cần thay đổi. sau khi chạy bước 7 thì tệp này sẽ được tự động lưu ở thư mục hiện tại.

In [18]:
# data_path = r'C:\Users\caotr\Documents\GitHub\python\HATEFULMEME_DETECTION\ANNOTATION\memedataset_group8_fortest.csv'
# image_path = r'C:\Users\caotr\Downloads\memedataset_highquality\group8copy'
image_path = r'C:\Users\caotr\Downloads\memedataset_highquality\random'
data_path = r'C:\Users\caotr\Documents\GitHub\python\HATEFULMEME_DETECTION\memedataset_random.csv'
annotated_file_name = 'random_annotated.csv'

BƯỚC 4: CHẠY DÒNG NÀY

In [19]:
dataset = pd.read_csv(data_path)
dataset['post_text'] = dataset['post_text'].astype(str)
dataset.head(5)

,post_id,post_text,post_time,post_image,post_image_description,post_url,Unnamed: 6,ocr_text
0,1474638926797903,"""Mình sẽ cố gắng tiết kiệm"" Tôi khi có tiền:",2023-09-06 20:31:23,NaN,['Có thể là hình ảnh về 1 người và văn bản'],https://m.facebook.com/groups/625122365082901/...,NaN,['Dibluishx']
1,3507067906284386,linh ơi là linh 🐧,2023-04-30 12:51:06,NaN,['Không có mô tả ảnh.'],https://m.facebook.com/groups/tiemtaphoameme/p...,NaN,"['Việt Tân', '1,2 triệu người thích Trang này,..."
2,918584235491787,Mong ad duyệt ạ!,2022-01-20 21:04:51,NaN,['Không có mô tả ảnh.'],https://m.facebook.com/groups/vietnammemes/per...,NaN,"['Nhìn mặt cũng chiến đấy', 'Thể đã bao giờ ăn..."
3,6541618795850378,tưởng trúng vé số :)),2023-05-16 13:06:33,NaN,['Không có mô tả ảnh.'],https://m.facebook.com/groups/3162507603761531...,NaN,"['SÁNG HÔM', 'ĐÓ , BỐ TÔI', 'CHO 200K', 'TIÊU ..."
4,6924221144256806,Hm,2023-09-03 12:50:01,NaN,"[""Có thể là hình ảnh về văn bản cho biết 'con ...",https://m.facebook.com/groups/3162507603761531...,NaN,"['con giân còn lộ to', 'mẹ đi chợ về vẫn mua',..."


BƯỚC 5: CHẠY DÒNG NÀY

In [26]:
image_list = []

annotated_load = pd.read_csv(annotated_file_name) if os.path.exists(annotated_file_name) else pd.DataFrame(columns=['post_id', 'label'])
annotated_count = 0

all_images = glob.glob(image_path + '/*.jpg')
for filename in all_images:
    path, name = filename, os.path.basename(filename).split('.')[0]
    data = dataset[dataset['post_id'] == int(name)]
    if int(name) in annotated_load['post_id'].values.tolist():
        annotated_count += 1
        continue
    image_list.append((data['post_text'].values[0], path, name))
print('Annotated: ', annotated_count)
labels = ['Definitely hateful', 'Probably hateful', 'Not sure', 'Probably not hateful', 'Definitely not hateful', 'Discard']


Annotated:  129


BƯỚC 6: CHẠY DÒNG NÀY và tiến hành gán nhãn

In [27]:

widget = jp.annotate(examples= image_list, labels=labels,
                     x_widget_factory=jp.widgets.x.multiple(
                         factories=[jp.widgets.x.html(),jp.widgets.x.image(width=500, height=500), jp.widgets.x.html()],
                         vertical_layout=True,
                     )
)
display(widget)

BƯỚC 7: CHẠY DÒNG NÀY. sau khi gán nhãn xong, dòng này sẽ giúp bạn lưu lại tiến trình gán nhãn của mình

Nếu bạn gán nhãn hết rồi thì không sao, còn nếu chưa hết, bạn có thể mở lại bằng cách chạy lại từ BƯỚC 5 hoặc chạy lại từ đầu cũng được

In [28]:
# save every 30s

def save():
    threading.Timer(60.0, save).start()
    result = []
    for i in widget.session.annotated_examples:
        text, id, label = i.x[0], i.x[2], i.y
        result.append((id, label))
    # save annotated data
    annotated = pd.DataFrame(result, columns=['post_id', 'label'])
    annotated['post_id'] = annotated['post_id'].astype(pd.Int64Dtype())
    annotated_data = pd.read_csv(annotated_file_name) if os.path.exists(annotated_file_name) else pd.DataFrame(columns=['post_id', 'label'])
    annotated_data = pd.concat([annotated_data, annotated]).drop_duplicates(subset=['post_id'], keep='last')
    annotated_data.to_csv(annotated_file_name, index=False)
save()



# result = []
# for i in widget.session.annotated_examples:
#     text, id, label = i.x[0], i.x[2], i.y
#     result.append((id, label))
# # save annotated data
# annotated = pd.DataFrame(result, columns=['post_id', 'label'])
# annotated['post_id'] = annotated['post_id'].astype(pd.Int64Dtype())
# annotated_data = pd.read_csv(annotated_file_name) if os.path.exists(annotated_file_name) else pd.DataFrame(columns=['post_id', 'label'])
# annotated_data = pd.concat([annotated_data, annotated]).drop_duplicates(subset=['post_id'], keep='last')
# annotated_data.to_csv(annotated_file_name, index=False)
# print('Annotated: ', len(annotated_data))
# print('remaining: ', len(all_images) - len(annotated_data))
